In [2]:
import os

In [4]:
os.chdir("../")
%pwd

'd:\\FSDS ineuron\\algorithms\\projects\\Mushrooms-Classification-project-with-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sibap865/Mushrooms-Classification-project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sibap865"
os.environ["MLFLOW_TRACKING_PASSWORD"]="16ce56e7b14879dd80d506be76f133c923585d91"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [13]:
from MushroomProject.constants import *
from MushroomProject.utils.common import read_yaml, create_directories,save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/sibap865/Mushrooms-Classification-project-with-MLflow.mlflow",
           
        )

        return model_evaluation_config

In [10]:
import os
from MushroomProject import logger
from sklearn.metrics import accuracy_score,confusion_matrix
from urllib.parse import urlparse 
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        acc = accuracy_score(actual,pred)
        logger.info("model accuracy_score is :",acc)
        mat = confusion_matrix(actual,pred)
        logger.info("model confusion matrix is :",mat)
        return acc
    


    def log_into_mlflow(self):
        logger.info("logging data to dagshub")
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            acc = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"accuracy": acc}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", acc)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestModel1")
            else:
                mlflow.sklearn.log_model(model, "model")

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-09-08 13:41:54,293: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-08 13:41:54,299: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-08 13:41:54,311: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-08 13:41:54,315: INFO: common: created directory at: artifacts]
[2023-09-08 13:41:54,319: INFO: common: created directory at: artifacts/model_evaluation]


[2023-09-08 13:41:55,024: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\hp\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'RandomForestModel1'.
2023/09/08 13:42:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestModel1, version 1
Created version '1' of model 'RandomForestModel1'.
